In [1]:
import pandas as pd
import numpy as np
import requests
df = pd.read_csv('深圳商铺数据.csv',error_bad_lines=False)

In [2]:
lst_lpname = df['loupanmingcheng'].value_counts().index.tolist()

lst_lploc = df['loupandizhi'].value_counts().index.tolist()

In [3]:
city = '深圳'
dict_location = {}
My_key = 'f654d644aeae0e1da847d7ca408f4370'
url_pre = 'https://restapi.amap.com/v3/geocode/geo?key={}&address={}&city={}'
counter = 0
for i in range(len(lst_lpname)):
    location = lst_lpname[i]
    url = url_pre.format(My_key, location, city)
    res = requests.get(url).json()
    if res['status'] == '0':
        location = lst_lploc[i]
        url = url_pre.format(My_key, location, city)
        res = requests.get(url).json()
        if res['status'] == '0':
            dict_location[lst_lpname[i]] = np.nan
            counter +=1
        else:
            try: 
                dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
                if counter % 250 == 0:
                    print('one batch(250) processed')
            except:
                dict_location[lst_lpname[i]] = np.nan
            
    else:
        try: 
            dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
            if counter % 250 == 0:
                print('one batch(250) processed')
        except:
            try:
                location = lst_lploc[i]
                url = url_pre.format(My_key, location, city)
                res = requests.get(url).json()
                if res['status'] == '0':
                    dict_location[lst_lpname[i]] = np.nan
                    counter +=1
                else:
                    try: 
                        dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
                        if counter % 250 == 0:
                            print('one batch(250) processed')
                    except:
                        dict_location[lst_lpname[i]] = np.nan
            except:
                dict_location[lst_lpname[i]] = np.nan
                
            
        counter +=1

one batch(250) processed
one batch(250) processed


In [4]:
df['location_coords'] = np.nan

counter = 0
for i in lst_lpname:
    counter += 1
    df_temp = df[df['loupanmingcheng'] == i]
    df_temp_index = df_temp.index.tolist()
    df.iloc[df_temp_index,22] = dict_location[i]
    if counter %50 == 0:
        print('A batch(50) is done')

A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done


In [5]:
df.head()

,zhuanrangfei,rental_price,pay_method,jianzhumianji,shiyongmianji,shengyuzuqi,zhuanrangfangshi,floor,zhuangxiu,building_type,...,loupandizhi,miankuan,jinshen,cenggao,elvator_number,service_price,function_list,release_date,url,location_coords
0,NaN,5800元/月,押一付一,30㎡,NaN,NaN,NaN,第1层(共1层),精装修\r\n,商业街商铺,...,\r\n 广东省深圳市...,4.00米,6.00米,5.00米,NaN,30元/月,"网络,停车位,中央空调,上水,下水,排烟,排污",2019/8/4 18:12:50,https://sz.shop.fang.com/zu/1_52627524.html,"114.273151,22.730692"
1,NaN,7800元/月,月付,60㎡,NaN,NaN,NaN,第2层(共4层),简单装修\r\n,购物中心/百货,...,\r\n 南新天桥旁边,NaN,NaN,NaN,NaN,NaN,"暖气,停车位,客梯",2019/7/14 20:34:11,https://sz.shop.fang.com/zu/1_52624559.html,"113.922966,22.541394"
2,NaN,7800元/月,月付,60㎡,NaN,NaN,NaN,第2层(共4层),简单装修\r\n,购物中心/百货,...,\r\n 南新天桥旁边,NaN,NaN,NaN,NaN,NaN,"暖气,停车位,客梯,货梯",2019/7/14 20:39:42,https://sz.shop.fang.com/zu/1_52624561.html,"113.923259,22.540876"
3,NaN,16500元/月,押二付一,22㎡,NaN,NaN,NaN,第1层(共30层),简单装修\r\n,购物中心/百货,...,\r\n 广东省深圳市...,6.50米,3.50米,5.00米,NaN,租金已包含,NaN,2019/7/16 13:16:37,https://sz.shop.fang.com/zu/1_52624766.html,"114.022864,22.623522"
4,NaN,2700元/月,押二付一,24㎡,NaN,NaN,NaN,第1层(共6层),简单装修\r\n,商业街商铺,...,\r\n 广东省深圳市...,5.50米,3.80米,2.80米,NaN,租金已包含,NaN,2019/7/16 13:36:29,https://sz.shop.fang.com/zu/1_52624768.html,"113.894641,22.780660"


In [6]:
df.to_csv('深圳商铺地址数据.csv', encoding = 'utf-8-sig',index = False)